#  Module 1 : Parts modales

**Description** : Le but de ce module est de mener un premier calcul des parts modales kilométriques i.e. les distances journalières moyennisées par mode et par motif.

**Durée estimée première partie** : 7 jours

**Objectifs spécifiques** :
- [ ] Sous-échantillonnage des résidents et visiteurs par canton (basé sur l’adresse déclarée)
- [ ] Rééchantillonnage des jours d’observation pour avoir un calendrier continue par usager
- [ ] Intégrer le détail des transit
- [ ] Distinguer de façon aussi systématique que possible les jours sans déplacement des jours
non-détectés et comparaison statistique au jours non-déplacé dans d’autres bases de
données
- [ ] Recodage des modes et motifs selon besoin des cantons
- [ ] Calcul liminaire des parts modales kilométriques et par déplacements
- [ ] Ajout des données d’équipement (e.g. type de motorisation principale du ménage)
- [ ] Documenter les hypothèses et limites du calcul liminaire des parts modales (e.g. aspects
saisonniers, échantillonnage, perte de signal, moyennisation des données longitudinales, ...)

**Résultats attendus** : Parts modales kilométriques par mode pour les résidents et visiteurs de chaque canton en vue du calcul des émissions carbone. Il doit être possible de calculer les parts modales en tenant compte des jours non-mobiles.

**Sous-échantillonnage** :
- Vaud : résident·es du canton
- Genève : résident·es du canton

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from shapely import geometry, ops
from shapely.geometry import MultiLineString, LineString, Point
import os
import concurrent.futures
from shapely.ops import unary_union
import xyt

import time

### Charger les données

In [ ]:
%%time
# Définir le CRS du projet (EPSG:4326 for WGS84)
target_crs = 'EPSG:4326'
print("CRS du projet: WGS84 \n")

#Charger les étapes
# Ask the user for input
choice = input("Do you want to load a sample or the full leg data frame? \n Enter 'sample' or 'full': ")

# Define the file paths
if choice.lower() == 'sample':
    file_path = '../Data/time_space_filters/legs_filtered_randsample.pkl'
elif choice.lower() == 'full':
    file_path = '../Data/time_space_filters/legs_filtered.pkl'
else:
    print("Invalid choice. Please enter 'sample' or 'full'.")

# Load the selected data frame
try:
    legs = pd.read_pickle(file_path)
    print("Fichier étape chargé")
except FileNotFoundError:
    print("File not found. Please check the file path.")
legs = gpd.GeoDataFrame(legs, geometry="geometry")

#Charger les activités
staypoints = pd.read_pickle('../Data/time_space_filters/staypoints_filtered.pkl').reset_index(drop=True)
staypoints = gpd.GeoDataFrame(staypoints, geometry="geometry")
print("Fichier activité chargé")

#Charger le questionnaire de la Vague 1 Mobilité
full_survey = pd.read_csv('../../Vague1/EPFL_vague1_v4.csv', low_memory=False)
dom =  pd.read_csv('../../Vague1/Localisation_domicile.csv', low_memory=False)
dom = gpd.GeoDataFrame(dom, geometry=gpd.points_from_xy(dom.dom_long, dom.dom_lat), crs=target_crs)
del dom['dom_long']
del dom['dom_lat']
print("Fichier questionnaire chargé")

#Charger les bases officielles pour le sous-échantillonage géographique https://opendata.swiss/de/dataset/vm-uvek-zones-2017/resource/29b98f2c-42f2-4e72-b8b1-a39500ed0ad0
TAZ = gpd.read_file('../../Vague1/Verkehrszonen_Schweiz_NPVM_2017_shp/Verkehrszonen_Schweiz_NPVM_2017.shp')
TAZ = TAZ[['ID_Agglo', 'N_Agglo', 'N_KT', 'ID_Gem', 'geometry']]
TAZ = TAZ.to_crs(crs=target_crs)
#repare anomalies
TAZ['geometry'] = TAZ['geometry'].buffer(0)
print("Fichier Zones de traffic chargé")

In [ ]:
#Q4_1_1_R : Combien avez-vous de voiture conventionnelle en état de fonctionnement dans votre ménage ?
#Q4_1_2_R : Combien avez-vous de voiture électrique/hybride en état de fonctionnement dans votre ménage ?
#Q5 : Quel est le type de motorisation de la voiture que vous utilisez le plus souvent ?
#Q6 : Quel est le type de motorisation de la deuxième voiture que vous utilisez le plus souvent ?
#Q7 : Pouvez-vous disposer d’une voiture du ménage quand vous le souhaitez ?
#Q8 : Vous arrive-t-il de vous faire prêter une voiture par des proches (ami·e·s, famille, etc.) pour vos propres déplacements ou ceux de votre ménage ?
#Q9 : Vous arrive-t-il d'utiliser un service d'autopartage (car-sharing, tel que Mobility ou Citiz) ?

survey = full_survey[['IDNO','canton_dep','AGGLO_CH_dom','Pays','Groupe', 'Weight', 'ID_COM', 'permis_auto', 'revenu', 'revenuFR','revenuCH','age','formation','Genre_actuel','KLASSE_ARE_dom', 'KLASSE_ARE_trav', 'pays_trav', 'Q4_1_1_R','Q4_1_2_R', 'Q5_R', 'Q6_R', 'Q7', 'Q8', 'Q9']].copy()

###  Formater les données

In [ ]:
staypoints['started_at'] = pd.to_datetime(staypoints['started_at'])
staypoints['finished_at'] = pd.to_datetime(staypoints['finished_at'])

legs['started_at'] = pd.to_datetime(legs['started_at'])
legs['finished_at'] = pd.to_datetime(legs['finished_at'])

staypoints.rename(columns={'IDNO':'user_id', 'id':'activity_id'}, inplace = True)
legs.rename(columns={'IDNO':'user_id', 'id':'leg_id'}, inplace = True)

staypoints['lon'] = staypoints.geometry.x
staypoints['lat'] = staypoints.geometry.y

In [ ]:
staypoints.dtypes

### Ajouter le *next activity_id* aux étapes

In [ ]:
# Sort 'points' and 'legs' by 'started_at' to ensure data is in chronological order
staypoints.sort_values(by=['user_id','started_at'], inplace=True, ignore_index=True)
legs.sort_values(by=['user_id','started_at'], inplace=True)

In [ ]:
legs = pd.merge(legs, staypoints[['activity_id', 'previous_leg_id']],
               left_on='leg_id', right_on='previous_leg_id', how='left')
legs.rename(columns={'activity_id':'next_activity_id'}, inplace=True)
del legs['previous_leg_id']

###  Ajouter la durée et la longueur des étapes

In [ ]:
%%time 
# Add length in meters
legs['length'] = legs.to_crs('EPSG:2056').length
# Add the duration in seconds
legs['duration'] = (legs['finished_at'] - legs['started_at']).dt.total_seconds()

In [ ]:
staypoints = pd.merge(staypoints, legs[['leg_id','length','duration']], how='left',left_on='previous_leg_id',right_on='leg_id')
staypoints.rename(columns={'length':'previous_length','duration':'previous_duration'}, inplace=True)
del staypoints['leg_id']
staypoints = pd.merge(staypoints, legs[['leg_id','length','duration']], how='left',left_on='next_leg_id',right_on='leg_id')
staypoints.rename(columns={'length':'next_length','duration':'next_duration'}, inplace=True)
del staypoints['leg_id']

### Extraire les aires géographiques et les sous-échantillons (Genève et Vaud)
Nous utilisons les zones de traffic du Modèle Voyageur de l'ARE.

We want to sample :
- all the residents of Canton de Genève
- all the activities that happen in Canton de Genève

In [ ]:
%%time

# Prompt the user for N_KT value
n_kt = input("De quelle région souhaitez-vous obtenir les résidents (par exemple, 'GE' ou 'VD'): ")

# Ask if we should include activities or just take the list of residents
include_activities = input("Voulez-vous inclure les activités dans la région {0} par des résidents d'autres cantons? (Oui/Non): ".format(n_kt))


# Unir les zones de trafic
N_KT_union = unary_union(TAZ[TAZ.N_KT == n_kt].geometry)

# Lister les résident·es du KT
list_residents_N_KT = dom.loc[dom.within(N_KT_union), 'IDNO'].tolist()

# Lister toutes les activités des résident·es du KT (incl. les activités hors KT)
list_act_N_KT_res = staypoints.loc[staypoints.user_id.isin(list_residents_N_KT), 'activity_id'].tolist()

if include_activities.lower() == 'oui':
    # Lister toutes les activités du KT (incl. activités des résident·es hors KT)
    activities_in_N_KT = staypoints.loc[staypoints.within(N_KT_union)]
    list_activities_in_N_KT = activities_in_N_KT.activity_id.tolist()

    # Merge the lists of activity IDs and drop duplicates
    list_staypoints_N_KT = list(set(list_act_N_KT_res) | set(list_activities_in_N_KT))
else:
    # Only take the list of residents
    list_staypoints_N_KT = list_act_N_KT_res
    activities_in_N_KT = staypoints.loc[(staypoints.activity_id.isin(list_act_N_KT_res))] #(staypoints.within(N_KT_union))
    activities_in_N_KT = activities_in_N_KT.dropna().loc[activities_in_N_KT.within(N_KT_union)]
    list_activities_in_N_KT = activities_in_N_KT.activity_id.tolist()

# Calculate the ratio of staypoints in the specified N_KT to total unique activities
ratio_staypoints_N_KT = len(list_staypoints_N_KT) / len(staypoints.activity_id.unique())

# Filter staypoints and legs for the specified N_KT
staypoints_N_KT = staypoints.loc[staypoints.activity_id.isin(list_staypoints_N_KT)].copy()
legs_N_KT = legs.loc[legs.next_activity_id.isin(list_staypoints_N_KT)].copy()

# Print the ratio of staypoints in the specified N_KT
print("\nRatio de points de séjour dans la région '{0}' par rapport aux activités uniques totales: {1:.2f}".format(n_kt, ratio_staypoints_N_KT))

# Print the normalized value counts of departure cantons for legs in the specified N_KT
print("\nValeurs normalisées des cantons d'origine pour les trajets dans la région '{0}':".format(n_kt))
print(legs_N_KT.canton_dep.value_counts(normalize=True))

# Print the number of activities recorded
print("\nNombre d'activités recensées :", len(legs_N_KT))

N_KT_union


In [ ]:
# Indiquer quelles activités se passent dans KT (booléan) par quel résident·es (booléan)

staypoints_N_KT['user_residence_in_KT'] = 0
staypoints_N_KT['activity_in_KT'] = 0

staypoints_N_KT.loc[staypoints_N_KT.activity_id.isin(list_act_N_KT_res), 'user_residence_in_KT'] = 1
staypoints_N_KT.loc[staypoints_N_KT.activity_id.isin(list_activities_in_N_KT), 'activity_in_KT'] = 1

In [ ]:
# Add the Municipality ID where the user lives
staypoints_N_KT = pd.merge(staypoints_N_KT, dom[['IDNO','dom_Typo_panel','dom_ID_COM']], left_on='user_id', right_on='IDNO', how='left')

# Add the Municipality ID where the activity occurs
staypoints_N_KT = gpd.sjoin(staypoints_N_KT, TAZ[['geometry','ID_Gem']], how='left', predicate="within")

# Rename columns for clarity
del staypoints_N_KT['index_right']
del staypoints_N_KT['IDNO']
staypoints_N_KT.rename(columns={'ID_Gem':'ID_Gem_activity', 'dom_Typo_panel':'Typo_user_home','dom_ID_COM':'ID_Gem_user_home'}, inplace=True)

# Recode the territorial typology
replacement_mapping = {
    'A': 'major_centers',
    'B': 'central_urban_areas',
    'C': 'urban_suburbs',
    'D': 'low_densities',
    'F': 'secondary_centers'
}

staypoints_N_KT['Typo_user_home'] = staypoints_N_KT['Typo_user_home'].replace(replacement_mapping)
staypoints_N_KT.loc[staypoints_N_KT.Typo_user_home.isna(), 'Typo_user_home'] = 'other'

# Sort values and parse data
staypoints_N_KT = staypoints_N_KT.sort_values(by=['user_id', 'started_date'])
staypoints_N_KT[['ID_Gem_activity', 'ID_Gem_user_home']] = staypoints_N_KT[['ID_Gem_activity', 'ID_Gem_user_home']].astype(int, errors='ignore')

In [ ]:
len(staypoints_N_KT)

### Visualize the data

In [ ]:
%autoreload
xyt.plot_gps(staypoints_N_KT.dropna()[:2000], geo_columns='geometry')

In [ ]:
staypoints_N_KT.columns

In [ ]:
staypoints_N_KT.activity_in_KT.value_counts()

In [ ]:
staypoints_N_KT.dropna().user_id.unique()

In [ ]:
df_ = staypoints.loc[staypoints.user_id.isin(['CH3181'])]
df_.head()

In [ ]:
staypoints_N_KT.loc[staypoints_N_KT.user_id.isin(['CH3181'])]

In [ ]:
df_.started_date.min()

In [ ]:
df_.started_date.max()

In [ ]:
df_.started_date.max() - df_.started_date.min()

In [ ]:
user_stat.columns

In [ ]:
user_stat = pd.read_csv('../Data/dumps_fors/UserStatistics.EPFL-Panel.2023-04-24--2023-06-05.csv', sep=';')

user_stat.loc[user_stat.IDNO.isin(['CH3181']),['IDNO','first_tracked_at', 'latest_confirmed_finished_at','confirmed_days_count','inactive_days_count','days_in_range_count']]

In [ ]:
#user_stat[user_stat.inactive_days_count !=0]

In [ ]:
user_stat

In [ ]:
metrics = xyt.GPSAnalytics()

In [ ]:
df = metrics.split_overnight(df_)
# DAILY USER_ID: Add user_ids per day
df.insert(
    1,
    "user_id_day",
    df["user_id"]
    + "_"
    + df.started_at.dt.year.astype(str)
    + df.started_at.dt.month.astype(str).str.zfill(2)
    + df.started_at.dt.day.astype(str).str.zfill(2),
)
df.head(3)

In [ ]:
def get_modal_distances(df):

    df['previous_length'] = df['previous_length'].astype(float)
    # Group by 'user_id_day', 'previous_mode', and 'previous_leg_id', then sum the distances
    grouped = df.groupby(['user_id','user_id_day', 'previous_mode'])['previous_length'].sum().reset_index()

    # Pivot the table to have modes as columns
    pivoted = grouped.pivot_table(
        index=['user_id','user_id_day'],
        columns='previous_mode',
        values='previous_length',
        aggfunc='sum'
    ).reset_index()

    # Resample to include missing days and fill NaNs with different values in different columns
    pivoted['date'] = pd.to_datetime(pivoted['user_id_day'].str[-8:])
    # Create a date range covering the entire date range for each ID
    date_ranges = pivoted.groupby('user_id')['date'].agg(['min', 'max']).reset_index()
    date_ranges['started_date'] = date_ranges.apply(lambda row: pd.date_range(row['min'], row['max'], freq='D'), axis=1)
    
    # Create a Cartesian product of IDs and date ranges
    cartesian = date_ranges.explode('started_date').reset_index(drop=True)
    #
    # Complete the original df with a continuous timeline
    pivoted_filled = pd.merge(pivoted, cartesian[['user_id', 'started_date']], how='outer', left_on=['user_id', 'date'], right_on=['user_id', 'started_date'])
    del pivoted_filled['started_date']

    
    # Fill missing values in the modes columns
    modes_columns = ['Mode::Airplane', 'Mode::Bicycle', 'Mode::Bus', 'Mode::Car', 'Mode::Train', 'Mode::Tram', 'Mode::Walk']
    for col in modes_columns:
        pivoted_filled[col] = pivoted_filled[col].fillna(0)  # Fill NaN values with 0 for the modes
    
    # Sort the resulting DataFrame
    pivoted_filled.sort_values(by=['user_id', 'date'], inplace=True)
    
    return pivoted_filled



In [ ]:
get_modal_distances(df)

In [ ]:
#Check if the code works
df.loc[(df.previous_mode == 'Mode::Walk') & (df.user_id_day == 'CH10223_20230502')]

### Drop some duplicates and Compute and save the distance indicators

In [ ]:
distances_N_KT = staypoints_N_KT[['activity_id', 'purpose', 'user_id', 
                              'previous_leg_id', 'scattered_tracking', 'started_at', 'finished_at', 'started_date', 
                              'user_residence_in_KT', 'activity_in_KT', 'Typo_user_home', 'ID_Gem_user_home', 'ID_Gem_activity']].copy()

In [ ]:
%%time

distances_N_KT = pd.merge(distances_N_KT, legs_N_KT[['detected_mode', 'mode', 'canton_dep',
       'low_quality_legs_1', 'low_quality_legs_2','next_activity_id', 'length', 'duration']].dropna(subset='next_activity_id'),
                       left_on='activity_id',right_on='next_activity_id', how='left')
del distances_N_KT['next_activity_id']

In [ ]:
distances_N_KT

### Reimport file for delivered to Canton an merge it to my file

In [ ]:
%%time 

pickle_file_path = "staypoints_filtered_with_sociodemo_withoutIDNO.pkl"
sociodemo = pd.read_pickle(pickle_file_path)

#Get all the pseudo user_ids
#pseudo = staypoints[['activity_id','user_id']].drop_duplicates().dropna().merge(
#    sociodemo[['id','jointure_g']].drop_duplicates().dropna(),
#    left_on='activity_id', right_on='id', how='left')[['user_id','jointure_g']].drop_duplicates()
#pseudo.rename(columns={'jointure_g':'user_id_pseudo', 'user_id':'IDNO'}, inplace=True)
#pseudo.to_pickle(f'pseudo_user_ids.pkl')
pseudo = pd.read_pickle('pseudo_user_ids.pkl')

IDNO_info = pd.read_pickle('days_of_observation.pkl')

In [ ]:
IDNO_info = IDNO_info.merge(pseudo, on='IDNO', how='left')
distances_N_KT = distances_N_KT.merge(IDNO_info, left_on='user_id', right_on='IDNO', how='left')

In [ ]:
%%time

distances_N_KT = distances_N_KT.dropna(subset='user_id_pseudo').merge(sociodemo[['jointure_g','gdr', 'age', 'educ_ch', 'educ_fr', 'prof', 'wgt_cant_t', 'wgt_agg_tr']], left_on='user_id_pseudo', right_on='jointure_g', how='left')
del distances_N_KT['jointure_g']
del sociodemo
del distances_N_KT['IDNO']

distances_N_KT.rename(columns={'gdr':'genre'}, inplace=True)

In [ ]:
distances_N_KT = distances_N_KT.dropna(subset='activity_id').sort_values(by=['user_id', 'started_date', 'length'])
distances_N_KT.drop_duplicates(subset='activity_id', keep='last', inplace=True)

In [ ]:
#loss compared to ariginal staypoint df
1- len(distances_N_KT) / len(staypoints_N_KT)

In [ ]:
#Cheeck for duplicates
distances_N_KT[distances_N_KT.duplicated('activity_id')]

### Temporary saves

In [ ]:
# Save whole file
distances_N_KT.to_pickle(f'temp_files/distances_N_KT.pkl')
legs_N_KT.to_pickle(f'temp_files/legs_N_KT.pkl')

In [ ]:
# Read the entire pickle file into memory
distances_N_KT = pd.read_pickle('temp_files/distances_N_KT.pkl')
legs_N_KT = pd.read_pickle('temp_files/legs_N_KT.pkl')

In [ ]:
distances_N_KT.columns

In [ ]:
import pandas as pd

# Custom aggregation function to calculate the ratio
def ratio(series):
    total_low_quality = series.sum()
    total_trips = len(series)
    return total_low_quality / total_trips

# Create an ExcelWriter object to save multiple DataFrames to different tabs
with pd.ExcelWriter('temp_files/distances_N_KT_panel.xlsx', engine='xlsxwriter') as writer:
    aggreg_cols = {
        'TYPO_territoire_de_residence': 'Typo_user_home',
        'KT_territoire_de_residence': 'canton_dep'
    }

    for sheet_name, aggreg_col in aggreg_cols.items():
        # Group by user_id, mode, and purpose
        grouped_df = distances_N_KT[['user_residence_in_N_KT', 'activity_in_N_KT', 'mode', 'purpose','length', 'user_id_pseudo', 'started_date', 'duration', 'low_quality_legs_1', 'low_quality_legs_2', 'genre', 'age', 'educ_fr', 'educ_ch', 'prof','wgt_cant_t','wgt_agg_tr'] + [aggreg_col]].groupby(['user_residence_in_N_KT', 'activity_in_N_KT', 'mode', 'purpose', 'user_id_pseudo'] + [aggreg_col])
                
        result_df = grouped_df.agg({
            'length': ['sum', 'count'],
            'started_date': 'nunique',
            'duration': 'sum',
            'low_quality_legs_1': ratio, 
            'low_quality_legs_2': ratio,
            'genre': 'first',
            'age': 'first',
            'educ_fr': 'first',
            'educ_ch': 'first',
            'prof': 'first',
            'wgt_cant_t': 'first',
            'wgt_agg_tr': 'first',
        }).reset_index()

        # Flatten the MultiIndex columns
        result_df.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in result_df.columns]

        # Save the result_df to a new Excel tab
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Add the additional sheet
    additional_sheet_df = distances_N_KT[['user_id_pseudo', 'Min Date', 'Max Date', 'Track duration', 'Number of observation', 'Missing Dates', 'Max Gap Between Days']].drop_duplicates().reset_index(drop=True).rename(columns={'Number of observation': 'Days with tracking'})
    additional_sheet_df.to_excel(writer, sheet_name='Jours_enquete_par_enquete', index=False)


In [ ]:
distances_N_KT.loc[(distances_N_KT.Typo_user_home == 'central_urban_areas') & (distances_N_KT.user_residence_in_N_KT == 1), 'mode'].unique()

In [ ]:
len(result_df.user_id_pseudo.unique())

In [ ]:
result_df['length_sum_per_day'] = result_df['length_sum'] / 1000 / result_df['started_date_nunique']
result_df[result_df.user_residence_in_N_KT == 1].groupby(['mode']).agg({'length_sum_per_day':'mean'})

In [ ]:
distances_N_KT[['user_id_pseudo','Min Date', 'Max Date', 'Track duration','Number of observation', 'Missing Dates', 'Max Gap Between Days']].drop_duplicates().reset_index(drop=True).rename(columns={'Number of observation':'Days with tracking'})

In [ ]:
distances_N_KT.columns